In [1]:
import mxnet as mx

/home/tapir/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
prefix="cnn_models/resnet20"
epoch=124

In [3]:
sym, args, auxs = mx.mod.module.load_checkpoint(prefix, epoch)

In [4]:
batch_size=1
val_iter=mx.image.ImageIter(batch_size=batch_size,data_shape=(3,32,32),path_imgrec="dataset/cifar10_val.rec")
testarray=val_iter.next().data
test_iter=mx.io.NDArrayIter(testarray,batch_size=batch_size)

In [5]:
def get_out(symbol_input,):
    mod=mx.mod.Module(symbol=symbol_input, context=mx.cpu(),label_names=None)
    mod.bind(for_training=False, data_shapes=test_iter.provide_data)
    mod.set_params(args, auxs)
    return mod.predict(eval_data=test_iter)
    
#result=mod.score(val_iter,[mx.metric.Accuracy()])
#print result

In [6]:
sym_in=sym.get_internals()['stage3_unit1_relu1_output']
sym_out_original=sym.get_internals()['stage3_unit1_conv1_output']

In [7]:
tensor_in=get_out(sym_in)

In [8]:
tensor_out_original=get_out(sym_out_original)

In [9]:
print tensor_in.shape
print tensor_out_original.shape

(1L, 32L, 16L, 16L)
(1L, 64L, 8L, 8L)


In [ ]:
sym_weights=sym.get_internals()['stage3_unit1_conv1_weight']
filters=args

In [14]:
filters= args["stage3_unit1_conv1_weight"]

In [31]:
filters.shape

(64L, 32L, 3L, 3L)

In [21]:
sym_out_test=mx.ndarray.Convolution(data=tensor_in, weight=filters, num_filter=64, kernel=(3, 3), stride=(2,2), pad=(1,1)
                                  ,no_bias=True, name="conv0")

In [25]:
import numpy as np

In [27]:
np.array_equal(sym_out_test.asnumpy(),tensor_out_original.asnumpy())

True

In [28]:
from sklearn.cluster import KMeans

In [47]:
filters_shaped=filters.reshape((64*32,9))

In [48]:
filters_shaped_back=filters_shaped.reshape((64,32,3,3))

In [49]:
np.array_equal(filters.asnumpy(),filters_shaped_back.asnumpy())

True

In [50]:
filters[0][0]


[[ 0.01016686 -0.01429639 -0.10081434]
 [ 0.09926124 -0.11094946 -0.05726987]
 [-0.0286475  -0.04705236  0.10059486]]
<NDArray 3x3 @cpu(0)>

In [51]:
filters_shaped_back[0][0]


[[ 0.01016686 -0.01429639 -0.10081434]
 [ 0.09926124 -0.11094946 -0.05726987]
 [-0.0286475  -0.04705236  0.10059486]]
<NDArray 3x3 @cpu(0)>

In [53]:
estimator=KMeans(n_clusters=128)

In [55]:
estimator.fit(filters_shaped.asnumpy())

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=128, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [60]:
filter_kmean_indexes=estimator.predict(X=filters_shaped.asnumpy())

In [62]:
filter_kmean_indexes.shape

(2048,)

In [63]:
filters_quantized=np.array([estimator.cluster_centers_[idx] for idx in filter_kmean_indexes])

In [67]:
filters_quantized.shape

(2048L, 9L)

In [65]:
filters_shaped.shape

(2048L, 9L)

In [66]:
filters_quantized=mx.nd.array(filters_quantized)

In [68]:
filters_quantized_reshaped=filters_quantized.reshape((64,32,3,3))

In [69]:
filters_quantized_reshaped[0][0]


[[-0.04077778 -0.13211453 -0.02785414]
 [ 0.04103499 -0.06585058 -0.02332328]
 [ 0.01637475 -0.07890984 -0.01297713]]
<NDArray 3x3 @cpu(0)>

In [70]:
clustered_result=mx.ndarray.Convolution(data=tensor_in, weight=filters_quantized_reshaped, num_filter=64, kernel=(3, 3), stride=(2,2), pad=(1,1)
                                  ,no_bias=True, name="conv0")

In [72]:
np.square(tensor_out_original.asnumpy()-clustered_result.asnumpy()).mean()

3.8051147

In [74]:
#test1
qmod=mx.mod.Module(symbol=sym, context=mx.gpu())
qmod.bind(for_training=False, data_shapes=test_iter.provide_data,label_shapes=val_iter.provide_label)


In [75]:
args["stage3_unit1_conv1_weight"]=filters_quantized_reshaped
qmod.set_params(args, auxs)
qmod.score(val_iter,[mx.metric.Accuracy()])

[('accuracy', 0.8778)]